In [45]:
using Base.Test

In [72]:
module OffsetIndices

import Base: +, -, abs, *, /, div, convert, ==, <=, >=, show, to_index


immutable OffsetIndex{O, T <: Integer} <: Integer
    i::T
    
    OffsetIndex{O, T}(x::T) where {O, T <: Integer} = new{O, T}(x + O)
end

const ZeroIndex{T <: Integer} = OffsetIndex{-1, T}
const OneIndex{T <: Integer} = OffsetIndex{0, T}

raw(x::OffsetIndex) = x.i
raw(x::Integer) = x

######

function show{O, T}(io::IO, oi::OffsetIndex{O, T})
    print(io, "|$(raw(oi)) (indexes as $(raw(oi) - O))|")
end

Base.eltype(::Type{OffsetIndex{O, T}}) where {O, T} = T
Base.eltype(oi::OffsetIndex) = eltype(typeof(oi))

# constructors and conversion
convert(::Type{OffsetIndex{O1, T1}}, x::OffsetIndex{O2, T2}) where {O1, O2, T1 <: Integer, T2 <: Integer} = 
    OffsetIndex{O1, T1}(T2(x))
convert(::Type{OffsetIndex{O}}, x::Integer) where {O} = convert(OffsetIndex{O, eltype(x)}, x)
convert(::Type{OffsetIndex{O}}, x::OffsetIndex) where {O} = convert(OffsetIndex{O, eltype(x)}, x)
convert(::Type{OffsetIndex{O, T}}, x::Integer) where {O, T <: Integer} = convert(OffsetIndex{O, T}, OneIndex{eltype(x)}(x))

convert(::Type{IT}, x::OffsetIndex{O, T}) where {IT <: Integer, O, T <: Integer} = IT(raw(x) - O)

Base.@pure pure_max(x1, x2) = x1 > x2 ? x1 : x2
Base.promote_rule(::Type{T1}, ::Type{OffsetIndex{O, T2}}) where {T1 <: Integer, O, T2} = T1
Base.promote_rule(::Type{OffsetIndex{O1, T1}}, ::Type{OffsetIndex{O2, T2}}) where {O1, O2, T1, T2} = OffsetIndex{pure_max(O1, O2), promote_type(T1, T2)}

to_index(I::OffsetIndex) = OneIndex(I)
to_index(I::OffsetIndex{0}) = raw(I)

######

end

OI = OffsetIndices

OffsetIndices

In [73]:
# an array to index into
x = [1, 2, 3, 4];

In [74]:
# Creating an index from a regular integer:
o1 = OI.OneIndex(1)
@show o1
@test x[o1] == 1

o1 = |1 (indexes as 1)|


Test Passed


In [75]:
# Creating a zero index still requires a one-based input
# (the moral here is that untyped integers are always
# implicitly one-based)
o0 = OI.ZeroIndex(1)
@show o0
@test x[o0] == 1

o0 = |0 (indexes as 1)|


Test Passed


In [76]:
# Conversion to an integer is always one-based
@test Int(OI.ZeroIndex(1)) == 1

Test Passed


In [77]:
# We can use reinterpret() to read from zero-based
# data without any explicit additions
zero_based_data = [0, 1, 2]
desired_type = OI.OneIndex{Int}
zero_based_indices = reinterpret(OI.ZeroIndex{Int}, zero_based_data)
@show zero_based_indices

# But converting those indices back to integers makes them one-indexed as usual
@test zero_based_indices == [1, 2, 3] 

zero_based_indices = OffsetIndices.OffsetIndex{-1,Int64}[|0 (indexes as 1)|, |1 (indexes as 2)|, |2 (indexes as 3)|]


Test Passed


In [78]:
# Converting from one offset to another preserves indexing:
o10 = convert(OI.OffsetIndex{10}, o1)
@show o10
@test x[o10] === x[o1]

o10 = |11 (indexes as 1)|


Test Passed


In [79]:
# ...but changes the raw representation:
@test OI.raw(o10) == 11

Test Passed
